# Метрические алгоритмы. Практика

В этом домашнем задании вы будете решать задачу классификации бутылок вина по различным характеристикам.

## Импорт библиотек, установка константных значений

In [1]:
import pandas as pd
import numpy as np

In [2]:
RANDOM_STATE = 42
TRAIN_SIZE = 0.75

In [3]:
rng = np.random.default_rng(RANDOM_STATE)

## Загрузка данных

In [4]:
from sklearn.datasets import load_wine

data = load_wine(as_frame=True)

X = data.data
y = data.target

## Задание 1

Посмотрите на количество классов и количество объектов каждого класса в датасете.

**Вопрос**:  
Сколько классов в задаче?

In [7]:
y.value_counts()

1    71
0    59
2    48
Name: target, dtype: int64

## Задание 2

Мы имеем дело с многоклассовой классификацией. Кроме того, классы не очень хорошо сбалансированы, поэтому для оценки качества модели метрика *accuracy* не подойдет.

Разбейте данные на тренировочную и тестовую части:  
тестовая часть - 25% от всех данных, зафиксируйте `random_state = RANDOM_STATE`.

In [29]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=RANDOM_STATE)

**Вопрос:**

Все ли признаки в данных одного масштаба?  
Проверьте это, выведя основные числовые характеристики матрицы `X_train` методом `describe` из библиотеки `pandas`.

По полученной таблице числовых характеристик определите, какой признак измеряется в сотнях?  
(если вариантов несколько, выберите признак с наибольшим средним значением).

In [11]:
X_train.describe().T

,count,mean,std,min,25%,50%,75%,max
alcohol,44.0,13.042955,0.847943,11.56,12.3700,13.050,13.7250,14.83
malic_acid,44.0,2.247727,1.008219,0.94,1.6375,1.765,2.7500,5.19
ash,44.0,2.375227,0.285881,1.36,2.2500,2.380,2.5125,3.23
alcalinity_of_ash,44.0,19.500000,4.303055,10.60,17.3250,20.000,22.0000,28.50
magnesium,44.0,98.613636,12.680723,80.00,87.7500,97.500,106.5000,126.00
total_phenols,44.0,2.315455,0.695030,1.30,1.7250,2.150,2.8750,3.88
flavanoids,44.0,2.072273,1.125815,0.57,1.2425,1.800,2.9825,5.08
nonflavanoid_phenols,44.0,0.352500,0.126732,0.13,0.2575,0.330,0.4325,0.63
proanthocyanins,44.0,1.608864,0.545925,0.42,1.2750,1.545,1.8700,2.96
color_intensity,44.0,5.483636,2.433656,1.95,3.3600,5.400,7.1250,10.80


## Задание 3

KNN требует того, чтобы все признаки были одного масштаба, поэтому масштабируйте данные при помощи `StandardScaler`.

Напоминаем, что обучать метод нужно только по тренировочным данным, а применять и к трейну, и к тесту.

После применения `StandardScaler` преобразуйте `X_train` и `X_test` к типу `pd.DataFrame`, названия новых объектов оставьте `X_train` и `X_test`.

In [30]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Обучите KNN с параметрами по умолчанию на тренировочных данных и сделайте предсказание на тесте.

In [31]:
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier()
knn_model.fit(X_train, y_train)
predict = knn_model.predict(X_test)

Будем измерять качество модели по метрике weighted $f1$-score.

Чтобы выбрать тип усреднения (micro, macro, weighted) в функции `f1_score` необходимо задать этот тип в гиперпараметре `average`.

Вычислите $f1$-score на тестовых данных.

**Вопрос:**

Чему равен $f1$-score на тестовых данных?

In [33]:
from sklearn.metrics import f1_score

f1_w = f1_score(y_test, predict, average='weighted')
round(f1_w,2)

0.96

## Задание 4

Попробуем улучшить модель.

Подберите оптимальное количество соседей (`n_neighbors`) из диапазона *от 3 до 30 с шагом 2* и веса соседей (`weights`):  
`uniform`, `distance` по кросс-валидации с тремя фолдами на тренировочных данных.

Используйте `GridSearchCV` и метрику `f1_weighted`.

In [40]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_neighbors' : np.arange(3,30, 2),
    'weights' : ['uniform', 'distance']
}
grid_model = GridSearchCV(KNeighborsClassifier(), param_grid=param_grid, cv=3)
grid_model.fit(X_train, y_train)
model = grid_model.best_estimator_

Возьмите best_estimator_, полученный при обучении GridSearchCV и с помощью него  
сделайте предсказание на тесте и вычислите метрику `f1_weighted`.

In [41]:
predict = model.predict(X_test)
f1_w = f1_score( y_test, predict, average='weighted')
f1_w

0.9550512333965844

**Вопрос:**

Удалось ли при помощи подбора гиперпараметров улучшить качество модели на тестовых данных?

## Задание 5

Выведите на экран матрицу ошибок.

Используйте модель с подобранными при помощи `GridSearch` гиперпараметрами.


**Вопрос:**  
По этой матрице определите, какие классы между собой путает модель?

In [42]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predict)

array([[15,  0,  0],
       [ 1, 16,  1],
       [ 0,  0, 12]])

## Бонус (эксперименты с LSH)

Скопируйте все функции из [ноутбука в уроке "Быстрый поиск соседей"](https://colab.research.google.com/drive/181MMOcTnzdMVzJr0pWzqtEG0-BV9BIHH).

In [ ]:
# ваш код здесь

При помощи `knn_search` найдите ближайших соседей к вину `X_test.iloc[0]` в **тренировочных** данных.

Обратите внимание, что функция `knn_search` принимает на вход `np.array`, а не `pd.DataFrame`. Поэтому переведите аргументы в `np.array`, приписав к необходимому объекту $X$: `X.values`.

In [ ]:
%%time

# ваш код здесь

Выведите на экран признаки объекта `X_test.iloc[0]` и признаки ближайшего найденного соседа.

In [ ]:
# ваш код здесь

**Вопрос:**

Можно ли сказать, что в тренировочных данных есть вино, почти такое же как `X_test.iloc[0]`? (все признаки почти одинаковые)

Какое расстояние между объектом запроса и первым ближайшим соседом?

Теперь найдите ближайшего соседа при помощи `approx_knn_search`.

In [ ]:
%%time

# ваш код здесь

Ближайший сосед при помощи KNN+LSH может быть найден не точно или не с первого запуска.  
Запустите последнюю ячейку несколько раз и убедитесь, что ближайший сосед находится верно за несколько запусков.

**Вопрос:**

Запустите `knn_search` и `appox_knn_search` несколько раз и сравните время запусков. Какой из подходов в этой задаче работает быстрее?

In [43]:
from sklearn.model_selection import LeaveOneOut

In [46]:
data = pd.DataFrame({'feature1' : [-1,1,1,0], 'feature2' : [1, -1, 1, 0], 'target' : [1,1,1,-1]})

In [50]:
loo = LeaveOneOut()
X = data.drop('target', axis=1)
y = data['target']
model = KNeighborsClassifier(metric='euclidean')
param_grid = {
    'n_neighbors' : [1,2,3]
}
grid_model = GridSearchCV(estimator=model, param_grid=param_grid, cv=loo)
grid_model.fit(X, y)

GridSearchCV(cv=LeaveOneOut(),
             estimator=KNeighborsClassifier(metric='euclidean'),
             param_grid={'n_neighbors': [1, 2, 3]})

In [51]:
grid_model.best_estimator_

KNeighborsClassifier(metric='euclidean', n_neighbors=3)